In [106]:
import spacy
import numpy as np
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jimmychen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jimmychen/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    return tokens

In [7]:
import json

text_data = []
with open("../../../data/courseroster/full_json.txt") as f:
    cornell_course_descriptions = json.load(f)

In [18]:
cornell_course_descriptions['CS']

[{'courseLength': None,
  'courseNumber': '1110',
  'courseTitle': 'Introduction to Computing Using Python',
  'description': 'Programming and problem solving using Python. Emphasizes principles of software development, style, and testing. Topics include procedures and functions, iteration, recusion, arrays and vectors, strings, an operational model of procedure and function calls, algorithms, exceptions, object-oriented programming, and GUIs (graphical user interfaces). Weekly labs provide guided practice on the computer, with staff present to help. Assignments use graphics and GUIs to help develop fluency and understanding.',
  'offered': ['Fall', 'Spring', 'Summer'],
  'outcomes': ' Be fluent in the use of procedural statements -assignments, conditional statements, loops, method calls- and  arrays. Be able to design, code, and test small Python programs that meet requirements expressed in English. This includes a basic understanding of top-down design. Understand the concepts of obj

In [123]:
course_to_LDA = {}
all_LDA_terms = set()
import regex as re
for dept in cornell_course_descriptions.keys():
    print("major:",dept)
    major_courses = cornell_course_descriptions[dept]
    text_data = []
    course_codes = []
    for i, course_data in enumerate(major_courses):
        course_number = course_data['courseNumber']
        course_title = course_data['courseTitle']
        course_desc = course_data['description']
        if not (course_desc is None or course_desc == [] or course_desc == ""):
            cleaned_course_desc = prepare_text_for_lda(course_desc)
            text_data.append(cleaned_course_desc)
    dictionary = corpora.Dictionary(text_data)
    dictionary.filter_extremes(no_below=2, no_above=0.5, keep_n=1000)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    if not(np.array(corpus).flatten().tolist() == []):
        lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=4, passes=10)
        for top in lda.print_topics(num_words=4):
            print(top)
            regex = r"[a-zA-Z]+"
            words = re.findall(regex,top[1])
            all_LDA_terms.update(words)
            

    else:
        print("Not enough classes for: ")
        print(dept)



major: VISST
(0, '0.031*"visual" + 0.019*"history" + 0.017*"latin" + 0.017*"american"')
(1, '0.025*"often" + 0.025*"textiles" + 0.019*"social" + 0.019*"southeast"')
(2, '0.042*"media" + 0.022*"theory" + 0.022*"digital" + 0.022*"concepts"')
(3, '0.025*"history" + 0.025*"world" + 0.019*"process" + 0.019*"global"')
major: JPLIT
Not enough classes for: 
JPLIT
major: MAE
(0, '0.027*"spacecraft" + 0.024*"design" + 0.019*"project" + 0.017*"include"')
(1, '0.030*"mechanical" + 0.022*"tissues" + 0.021*"systems" + 0.017*"mechanics"')
(2, '0.039*"course" + 0.034*"systems" + 0.020*"mechanical" + 0.018*"design"')
(3, '0.029*"project" + 0.027*"engineering" + 0.021*"students" + 0.019*"computational"')
major: AEM
(0, '0.033*"business" + 0.011*"develop" + 0.010*"principles" + 0.010*"semester"')
(1, '0.016*"economic" + 0.013*"management" + 0.012*"decision" + 0.012*"economics"')
(2, '0.022*"topics" + 0.018*"financial" + 0.015*"management" + 0.013*"include"')
(3, '0.017*"research" + 0.012*"business" + 0.0

(0, '0.040*"topics" + 0.034*"plant" + 0.034*"plants" + 0.034*"systems"')
(1, '0.039*"number" + 0.036*"course" + 0.029*"plants" + 0.027*"offered"')
(2, '0.102*"students" + 0.046*"plhrt" + 0.037*"preparation" + 0.037*"campus"')
(3, '0.043*"management" + 0.033*"gardens" + 0.033*"techniques" + 0.028*"production"')
major: POLSH
(0, '0.310*"working" + 0.310*"continue" + 0.310*"enhance" + 0.035*"develop"')
(1, '0.692*"develop" + 0.077*"poland" + 0.077*"enhance" + 0.077*"continue"')
(2, '0.692*"poland" + 0.077*"develop" + 0.077*"enhance" + 0.077*"working"')
(3, '0.200*"develop" + 0.200*"poland" + 0.200*"enhance" + 0.200*"working"')
major: NBAY
(0, '0.157*"business" + 0.082*"value" + 0.051*"markets" + 0.051*"market"')
(1, '0.085*"technology" + 0.085*"design" + 0.059*"firms" + 0.059*"future"')
(2, '0.088*"decisions" + 0.068*"systems" + 0.068*"tools" + 0.068*"useful"')
(3, '0.090*"prepare" + 0.089*"organizations" + 0.089*"better" + 0.018*"business"')
major: ORIE
(0, '0.035*"topics" + 0.026*"resea

(0, '0.041*"political" + 0.031*"western" + 0.021*"christian" + 0.021*"notions"')
(1, '0.037*"history" + 0.028*"course" + 0.023*"texts" + 0.023*"century"')
(2, '0.032*"middle" + 0.025*"gender" + 0.025*"simple" + 0.025*"country"')
(3, '0.030*"depictions" + 0.025*"attention" + 0.025*"heroic" + 0.025*"middle"')
major: HEBRW
(0, '0.131*"issues" + 0.131*"preparation" + 0.072*"israelite" + 0.072*"original"')
(1, '0.178*"biblical" + 0.123*"ancient" + 0.068*"commentaries" + 0.068*"close"')
(2, '0.243*"writing" + 0.135*"grammatical" + 0.135*"discussion" + 0.135*"israeli"')
(3, '0.170*"israeli" + 0.170*"stories" + 0.094*"discussion" + 0.094*"language"')
major: VTMED
(0, '0.020*"student" + 0.019*"medicine" + 0.017*"animal" + 0.015*"clinical"')
(1, '0.024*"animal" + 0.022*"animals" + 0.021*"clinical" + 0.015*"management"')
(2, '0.017*"clinical" + 0.017*"practice" + 0.015*"lectures" + 0.014*"equine"')
(3, '0.032*"faculty" + 0.025*"research" + 0.021*"specific" + 0.019*"projects"')
major: PALI
Not eno

(0, '0.025*"climbing" + 0.020*"students" + 0.020*"class" + 0.017*"learn"')
(1, '0.025*"class" + 0.019*"learn" + 0.017*"course" + 0.016*"skills"')
(2, '0.025*"training" + 0.018*"course" + 0.016*"physical" + 0.014*"include"')
(3, '0.031*"class" + 0.020*"course" + 0.019*"dance" + 0.017*"learn"')
major: ILRLE
(0, '0.086*"theory" + 0.086*"human" + 0.059*"resource" + 0.059*"economic"')
(1, '0.032*"course" + 0.032*"including" + 0.032*"demand" + 0.031*"inequality"')
(2, '0.106*"education" + 0.081*"economic" + 0.081*"demand" + 0.081*"great"')
(3, '0.108*"course" + 0.075*"including" + 0.042*"management" + 0.042*"training"')
major: SNLIT
Not enough classes for: 
SNLIT
major: GOVT
(0, '0.025*"equality" + 0.021*"power" + 0.017*"peace" + 0.015*"seminar"')
(1, '0.024*"study" + 0.019*"american" + 0.017*"social" + 0.015*"students"')
(2, '0.020*"issues" + 0.019*"students" + 0.018*"science" + 0.016*"research"')
(3, '0.020*"politics" + 0.017*"students" + 0.016*"authority" + 0.013*"contemporary"')
major: B

(0, '0.054*"topics" + 0.053*"biomg" + 0.035*"research" + 0.029*"include"')
(1, '0.062*"biology" + 0.060*"genetic" + 0.035*"course" + 0.032*"research"')
(2, '0.065*"course" + 0.044*"research" + 0.044*"genetic" + 0.044*"cells"')
(3, '0.046*"topics" + 0.040*"protein" + 0.035*"structure" + 0.031*"molecular"')
major: PORT
(0, '0.023*"portuguese" + 0.023*"native" + 0.023*"spanish" + 0.023*"level"')
(1, '0.058*"brazilian" + 0.047*"native" + 0.047*"portuguese" + 0.036*"range"')
(2, '0.027*"brazilian" + 0.023*"topics" + 0.023*"culture" + 0.023*"portuguese"')
(3, '0.137*"topics" + 0.072*"specific" + 0.072*"study" + 0.072*"courses"')
major: SHUM
(0, '0.033*"collections" + 0.027*"political" + 0.014*"roman" + 0.014*"called"')
(1, '0.030*"claim" + 0.026*"political" + 0.021*"groups" + 0.021*"state"')
(2, '0.058*"moral" + 0.024*"follow" + 0.024*"obedience" + 0.013*"explore"')
(3, '0.014*"least" + 0.014*"straightforward" + 0.014*"activity" + 0.014*"resentment"')
major: NMI
Not enough classes for: 
NMI


(0, '0.019*"entrepreneurial" + 0.015*"management" + 0.014*"leadership" + 0.014*"industry"')
(1, '0.025*"marketing" + 0.015*"industry" + 0.014*"hospitality" + 0.012*"management"')
(2, '0.025*"estate" + 0.016*"analysis" + 0.014*"financial" + 0.013*"investment"')
(3, '0.023*"information" + 0.019*"management" + 0.016*"business" + 0.014*"design"')
major: PLBRG
(0, '0.101*"genetics" + 0.036*"passed" + 0.023*"students" + 0.023*"research"')
(1, '0.024*"students" + 0.024*"genetics" + 0.023*"introductory" + 0.023*"ornamentals"')
(2, '0.150*"candidates" + 0.150*"thesis" + 0.150*"research" + 0.096*"passed"')
(3, '0.064*"students" + 0.044*"genetics" + 0.033*"reviews" + 0.033*"concepts"')
major: GRAD
Not enough classes for: 
GRAD
major: PSYCH
(0, '0.037*"graduate" + 0.024*"social" + 0.023*"topics" + 0.023*"discussion"')
(1, '0.044*"imagination" + 0.044*"human" + 0.021*"research" + 0.021*"environmental"')
(2, '0.041*"psychology" + 0.041*"research" + 0.036*"language" + 0.028*"social"')
(3, '0.040*"hip

In [124]:
print(len(all_LDA_terms))
print(all_LDA_terms)
outF = open("LDA_terms_top_4.txt", "w")
for word in list(all_LDA_terms):
    outF.write(word)
    outF.write("\n")
outF.close()

714
{'genetic', 'disciplines', 'learning', 'biblical', 'movements', 'marketing', 'trade', 'performance', 'technology', 'textiles', 'peoples', 'understand', 'm', 'continue', 'asian', 'tropical', 'protein', 'rights', 'grammatical', 'electronic', 'internal', 'examples', 'europe', 'section', 'employee', 'native', 'authors', 'understanding', 'objects', 'project', 'studying', 'readings', 'politics', 'variety', 'islamic', 'anthropology', 'focus', 'conversation', 'various', 'marvelous', 'mineral', 'properties', 'clinic', 'library', 'authority', 'process', 'america', 'industrial', 'music', 'middle', 'department', 'basic', 'analog', 'texts', 'immigrant', 'russa', 'spatial', 'product', 'statistics', 'terminology', 'cases', 'mediation', 'using', 'animal', 'speakers', 'discuss', 'listening', 'japan', 'follow', 'ethics', 'social', 'involves', 'architectural', 'exoplanets', 'natural', 'summary', 'supervisors', 'registration', 'define', 'science', 'importance', 'summer', 'seminars', 'queer', 'justice'

In [ ]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
dictionary.filter_extremes(no_below=5, no_above=0.3, keep_n=1000)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('lda_corpus.pkl', 'wb'))
dictionary.save('lda_dictionary.gensim')

In [84]:
# import gensim
# NUM_TOPICS = 10
# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# ldamodel.save('lda_model_5.gensim')
# topics = ldamodel.print_topics(num_words=5)
# for topic in topics:
#     print(topic)

In [93]:
lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=25, passes=10)

# Prints the topics.
for top in lda.print_topics(num_words=6):
    print(top)
print("")

ValueError: cannot compute LDA over an empty collection (no terms)

In [67]:
lda_corpus = lda[corpus]

In [73]:
from itertools import chain
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))
threshold = sum(scores)/len(scores)
print(threshold)
print("")
print(len(lda_corpus))
print(len(text_data))
print(lda_corpus[1])

0.4367572792593843

83
83
[(0, 0.010212973), (1, 0.58802116), (2, 0.40176585)]


In [69]:
cluster1 = [j for i,j in zip(lda_corpus,text_data) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,text_data) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,text_data) if i[2][1] > threshold]

IndexError: list index out of range

In [70]:
cluster1 = []
cluster2 = []
cluster3 = []
for i,j in zip(lda_corpus,text_data):
    for cluster, score in i:
        if score > threshold:
            if cluster == 0:
                cluster1.append(j)
            elif cluster == 1:
                cluster2.append(j)
            elif cluster == 2:
                cluster3.append(j)
    

In [71]:
print(len(cluster1))
print(len(cluster2))
print(len(cluster3))

28
28
28
